In [1]:
!pip install streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from pandas.errors import ParserError
import pickle

# Load the merged and labeled data, trying different delimiters
try:
    data = pd.read_csv('aditya.csv', sep=',')  # Try default comma first
except ParserError:
    try:
        data = pd.read_csv('aditya.csv', sep=';')  # Try semicolon
    except ParserError:
        data = pd.read_csv('aditya.csv', sep='\t')  # Try tab

# Print the column names to verify if 'domain' exists
print(data.columns)


Index(['domain', 'label'], dtype='object')


In [3]:
# Separate features (domain names) and labels
X = data['domain'].values  # Replace 'domain' with the correct column name if needed
y = data['label'].values

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Pad sequences to have the same length
maxlen = max([len(seq) for seq in X_sequences])
X_padded = pad_sequences(X_sequences, maxlen=maxlen)

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=maxlen))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=1, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6731/6731 ━━━━━━━━━━━━━━━━━━━━ 1312s 194ms/step - accuracy: 0.9520 - loss: 0.1116 - val_accuracy: 0.9986 - val_loss: 0.0051


In [5]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

# Save the model and tokenizer
model.save('model.h5')

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

2104/2104 ━━━━━━━━━━━━━━━━━━━━ 94s 45ms/step - accuracy: 0.9986 - loss: 0.0054


Test Accuracy: 1.00


In [ ]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the model and tokenizer
model = load_model('model.h5')

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define the maxlen (the length you used during training)
maxlen = 50  # Replace this with the actual maxlen used in your training

# Streamlit app setup
st.title("Domain Legitimacy Prediction")
st.write("Enter a domain name to check if it's legitimate or non-legitimate.")

# Get user input
domain_input = st.text_input("Domain name:")

if st.button("Predict"):
    if domain_input:
        # Preprocess the input
        new_sequences = tokenizer.texts_to_sequences([domain_input])
        new_padded = pad_sequences(new_sequences, maxlen=maxlen, padding='post')

        # Predict
        prediction = model.predict(new_padded)
        result = 'Non-Legitimate' if prediction > 0.5 else 'Legitimate'

        # Show result
        st.write(f"The domain '{domain_input}' is predicted to be **{result}**.")
    else:
        st.write("Please enter a domain name.")


In [ ]:
from pyngrok import ngrok

# Start the Streamlit app
!streamlit run app.py &

# Create a tunnel to the Streamlit app
public_url = ngrok.connect(port='8501')
print(f'Public URL: {public_url}')





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.138.167.154:8501

